In [2359]:
import numpy as np
import math
import networkx as nx
import random

In [2360]:
class Tank():
    def __init__(self, tank_id, current_load, max_load, consumption_rate):
        self.id = tank_id
        self.load = current_load
        self.max_load = max_load
        self.rate = consumption_rate
   
    def fill(self):
        self.load = self.max_load    
        
    def partial_fill(self, fill_percentage):
        self.load = self.load + self.max_load * fill_percentage
    
    def tank_extra_capacity(self):
        return(self.max_load - self.load)
        
       
        

In [2361]:
class Truck():
    def __init__(self, truck_id, current_load, max_load, current_position, load_fractions_deliverable):
        self.id = truck_id
        self.load = current_load
        self.max_load = max_load
        self.pos = current_position
        self.fractions = load_fractions_deliverable
        
    def fill(self):
        self.load = self.max_load
        
    def deliver(self, fraction_id: int):
        self.load = self.load - self.fractions[fraction_id] * self.max_load
    
    def possible_delivery_quantities(self, tank_extra_capacity):
        all_delivery_quantities = self.load * self.fractions
        return(all_delivery_quantities[ all_delivery_quantities <= tank_extra_capacity].astype(list))
       
    

In [2362]:
class System():
    def __init__(self, tanks, trucks, adjacency_matrix, weights_matrix):
        self.tanks = tanks
        self.trucks = trucks
        self.graph = adjacency_matrix
        self.weights = weights_matrix
        self.k = len(trucks)
        self.n = len(tanks)
        self.s = self.state()
        
    def truck_loads(self):
        return([self.trucks[i].load for i in range(self.k)])
    
    def truck_positions(self):
        return([self.trucks[i].pos for i in range(self.k)])
    
    def tank_loads(self):
        return([self.tanks[i].load for i in range(self.n)])
    
    def state(self):
        #[ positions, truck-loads, tank-loads]
        s = [self.truck_positions(), self.truck_loads(), self.tank_loads()]
        return(s)
    
    def update_state(self):
        self.s = self.state()
        
    def random_action(self, seed = None):
        if seed != None:
            random.seed(seed)
            
        rewards = 0     
            
        # Choose a position for each truck randomly
        
        #CORRE GIR EL CODIGO, PLANTEAMIENTO PARA QUE FUNCIONE
        #possible_positions_index = np.isin(self.graph, 1)
        #possible_positions = np.where(possible_positions_index)
        #print("possible_positions:", possible_positions)
        for i in range(self.k):
            old_position = trucks[i].pos
            print("truck pos: ", old_position)
            possible_positions_index = np.isin(self.graph[old_position], 1)
            possible_positions = np.where(possible_positions_index)
            #random.randint(0,len(possible_positions[0])-1)
            print("nº of possible positions", len(possible_positions_index))
            new_position = random.randrange(len(possible_positions_index)-1 )
            print("new position: ",new_position)
            self.trucks[i].pos = new_position
            print("possible_positions:", possible_positions)
            
            # Update rewards due to oil costs (transport/km)
            rewards = rewards - self.weights[old_position][new_position]

            
            
        # Choose a new (possible) load delivery for each truck to the new tank (position)
        # and update the tank's load after deliverying the chosen quantity.
        for i in range(self.k):
            truck_pos = self.trucks[i].pos
            print("truck_pos: ", truck_pos)
            if False:
                continue;
            else:
                current_truck = trucks[i]
                current_tank = tanks[truck_pos]
                current_extra_tank_capacity = current_tank.tank_extra_capacity()
                possible_delivery_quantities = current_truck.possible_delivery_quantities(current_extra_tank_capacity)
                print("Possible delivery quantities: ", possible_delivery_quantities)
                if possible_delivery_quantities.size == 0:
                    print(f"Truck {trucks[i].id} in tank {trucks[i].pos} does not deliver")
                else:
                    delivery_quantity = np.random.choice(possible_delivery_quantities)
                    current_tank.load = current_tank.load + delivery_quantity
                    print(f"Truck {trucks[i].id} in tank {trucks[i].pos} delivers {delivery_quantity} units")
                
                    rewards = rewards - delivery_quantity
            
        self.update_state()
        return(rewards)
            

            


In [2363]:
random.randint(0,100)
print(np.__version__)



1.13.3


In [2364]:
# A simple system's adjacency matrix
n = 5

def simple_graph(n: int):    
    A = np.zeros((n,n))
    A[0,0:n] = 1
    A = A.astype(int)
    return(A)

A = simple_graph(n+1)

print(A)

G = nx.from_numpy_matrix(np.array(A)) 
nx.draw(G, with_labels=True)

A[1][1] = 1
A[1][2] = 1
A[2] = 1
print(A)
A[1]

[[1 1 1 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]
[[1 1 1 1 1 1]
 [0 1 1 0 0 0]
 [1 1 1 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


array([0, 1, 1, 0, 0, 0])

In [2365]:
possible_positions_index = np.isin(A[1], 1)
possible_positions = np.where(possible_positions_index)
print(possible_positions,random.randrange(len(possible_positions) +1 ) )

(array([1, 2]),) 1


In [2366]:
# Tanks' information
n = 5 
tank_ids = list(range(1,n+1))
tank_max_loads =  np.array([100, 100., 200., 300., 400.])
tank_current_loads =  np.array([50, 60., 120., 150., 300.])
tank_consumption_rates =  np.array([2] * n)


In [2367]:
# Trucks' information
k = 2
truck_ids = list(range(k))
truck_max_loads = np.array([20., 50.])
truck_current_loads = truck_max_loads.copy()
truck_current_positions =  np.array([0] * k)
#truck_fractions_deliverable =  np.array([1.] * k) # we for now we only allow to deliver all the content of the truck
truck_fractions_deliverable =  np.array([ np.array([0.5, 1.]), 
                                          np.array([0.5, 1.])
                                        ]) # we for now we only allow to deliver all the content of the truck


In [2368]:
# System's information
graph = simple_graph(n+1)
tanks = [Tank( tank_id, current_load, max_load, consumption_rate ) 
         for  tank_id, current_load, max_load, consumption_rate in 
         zip( tank_ids, tank_current_loads, tank_max_loads, tank_consumption_rates)]
trucks = [Truck( truck_id, current_load, max_load, current_position, load_fractions_deliverable ) 
         for  truck_id, current_load, max_load, current_position, load_fractions_deliverable in 
         zip(truck_ids, truck_current_loads, truck_max_loads, truck_current_positions, 
             truck_fractions_deliverable)]

def simple_weights(n:int, w: float):    
    W = np.full((n,n), np.inf)
    W[0,:] = w
    return(W)
w =  np.array([0, -20., -10., -30., -50.5, -45.])

weights_matrix = simple_weights(n+1, w)

toy_system = System(tanks, trucks, graph, weights_matrix)


In [2369]:
for i in range(n):
    print(tanks[i].id) 

#print(toy_system.truck_loads())
print(toy_system.tank_loads())
print(toy_system.truck_positions())

print(toy_system.state())
print(toy_system.weights)
#POSITION 'n' is the position of the Charge location (node 0 in the plot) ???? clarify notation and code

1
2
3
4
5
[50.0, 60.0, 120.0, 150.0, 300.0]
[0, 0]
[[0, 0], [20.0, 50.0], [50.0, 60.0, 120.0, 150.0, 300.0]]
[[  0.  -20.  -10.  -30.  -50.5 -45. ]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]
 [  inf   inf   inf   inf   inf   inf]]


In [2370]:
seed = None
print(toy_system.state())
toy_system.random_action(seed)
print(toy_system.state())

[[0, 0], [20.0, 50.0], [50.0, 60.0, 120.0, 150.0, 300.0]]
truck pos:  0
nº of possible positions 6
new position:  0
possible_positions: (array([0, 1, 2, 3, 4, 5]),)
truck pos:  0
nº of possible positions 6
new position:  1
possible_positions: (array([0, 1, 2, 3, 4, 5]),)
truck_pos:  0
Possible delivery quantities:  [10.0 20.0]
Truck 0 in tank 0 delivers 10.0 units
truck_pos:  1
Possible delivery quantities:  [25.0]
Truck 1 in tank 1 delivers 25.0 units
[[0, 1], [20.0, 50.0], [60.0, 85.0, 120.0, 150.0, 300.0]]


In [2371]:
print(toy_system.graph)

[[1 1 1 1 1 1]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]
 [0 0 0 0 0 0]]


In [2372]:
[trucks[j].load for j in range(k)]

[20.0, 50.0]

In [2373]:
k

2

In [2374]:
truck_current_positions 

array([0, 0])

In [2375]:

trucks[0].possible_delivery_quantities()

TypeError: possible_delivery_quantities() missing 1 required positional argument: 'tank_extra_capacity'